<a href="https://colab.research.google.com/github/Wadaken43/DeepLearning/blob/main/yolov3_origin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from torch.nn.modules.batchnorm import BatchNorm2d
import torch
import torch.nn as nn
import torch.nn.functional as F

ネットワークの構築

In [23]:
#残差ブロック
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, in_channels // 2, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels // 2)
        self.conv2 = nn.Conv2d(in_channels // 2, in_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(in_channels)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.leaky_relu(out, inplace=True)
        out = self.conv2(out)
        out = self.bn2(out)
        out = F.leaky_relu(out, inplace=True)
        out += residual
        return out


 #残差ブロック
  #   def MakeResidualBlock(self,fn):
  #       block = nn.Sequential(nn.Conv2d(fn , int(fn/2) , 1 , 1), 
  #                         nn.BatchNorm2d(int(fn/2)),
  #                         nn.LeakyReLU(),
  #                         nn.Conv2d(int(fn/2) , fn , 3 , 1 ,  1),
  #                         nn.BatchNorm2d(fn),
  #                         nn.LeakyReLU(),
  #  )


In [24]:
class YOLOv3(nn.Module):
 def __init__(self,in_channel=3, class_n = 80):
   super(YOLOv3 , self).__init__()
   self.class_n = class_n

   #darknet-53
   self.first_block = nn.Sequential(
       nn.conv2d(in_channel, 32, kernel_size=3, stride=1, padding=1),
       nn.BatchNorm2d(32),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
       nn.conv2d(32, 64, 3, 2, 1),
       nn.BatchNorm2d(64),
       nn.LeakyReLU(0.1, True),   
   )
   
   self.res_block1 = ResidualBlock(64)
   self.conv1 = nn.conv2d(64, 128, 3, 2, 1)
   
   self.res_block2 = nn.Sequential(
       ResidualBlock(128),
       ResidualBlock(128),
   )
   self.conv2 = nn.conv2d(128, 256, 3, 2, 1)
   
   self.res_block3 = nn.Sequential(
       ResidualBlock(256),
       ResidualBlock(256),
       ResidualBlock(256),
       ResidualBlock(256),
       ResidualBlock(256),
       ResidualBlock(256),
       ResidualBlock(256),
       ResidualBlock(256),
       ResidualBlock(256),
   )       
   self.conv3 = nn.conv2d(256, 512, 3, 2, 1)

   self.res_block4 = nn.Sequential(
       ResidualBlock(512),
       ResidualBlock(512),
       ResidualBlock(512),
       ResidualBlock(512),
       ResidualBlock(512),
       ResidualBlock(512),
       ResidualBlock(512),
       ResidualBlock(512),
   )   
   self.conv4 = nn.conv2d(512, 1024, 3, 2, 1)

   self.res_block5 = nn.Sequential(
       ResidualBlock(1024),
       ResidualBlock(1024),
       ResidualBlock(1024),
       ResidualBlock(1024),
   )  

   self.ConvBlock = nn.Sequential(
       nn.Conv2d(1024, 512, 1, 1, 1),
       nn.BatchNorm2d(512),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
       nn.Conv2d(512, 1024, 3, 1, 1),
       nn.BatchNorm2d(1024),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
  
       nn.Conv2d(1024, 512, 1, 1, 1),
       nn.BatchNorm2d(512),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
       nn.Conv2d(512, 1024, 3, 1, 1),
       nn.BatchNorm2d(1024),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),

       nn.Conv2d(1024, 512, 1, 1, 1),
       nn.BatchNorm2d(512),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
       nn.Conv2d(512, 1024, 3, 1, 1),
       nn.BatchNorm2d(1024),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
   )

   #FPN構造を構築するための
   #スケール3
   self.scale3_output = nn.Conv2d(1024, (3*(4+1+self.class_n)), 1, 1, 1)

   #スケール2
   self.scale2_UpSample = nn.Conv2d(1024, 256, 1, 1, 1)
   
   self.scale2_ConvBlock = nn.Sequential(
       nn.Conv2d(768, 256, 1, 1, 1),
       nn.BatchNorm2d(256),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
       nn.Conv2d(256, 512, 3, 1, 1),
       nn.BatchNorm2d(512),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
  
       nn.Conv2d(512, 256, 1, 1, 1),
       nn.BatchNorm2d(256),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
       nn.Conv2d(256, 512, 3, 1, 1),
       nn.BatchNorm2d(512),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),

       nn.Conv2d(512, 256, 1, 1, 1),
       nn.BatchNorm2d(256),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
       nn.Conv2d(256, 512, 3, 1, 1),
       nn.BatchNorm2d(512),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
   )
   self.scale2_output = nn.Conv2d(512, (3*(4+1+self.class_n)), 1, 1, 1)

   #スケール1
   self.scale1_UpSample = nn.Conv2d(512, 128, 1, 1, 1)
   self.scale1_ConvBlock = nn.Sequential(
       nn.Conv2d(384, 128, 1, 1, 1),
       nn.BatchNorm2d(128),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
       nn.Conv2d(128, 256, 3, 1, 1),
       nn.BatchNorm2d(256),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
  
       nn.Conv2d(256, 128, 1, 1, 1),
       nn.BatchNorm2d(128),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
       nn.Conv2d(128, 256, 3, 1, 1),
       nn.BatchNorm2d(256),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),

       nn.Conv2d(256, 128, 1, 1, 1),
       nn.BatchNorm2d(128),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
       nn.Conv2d(128, 256, 3, 1, 1),
       nn.BatchNorm2d(256),
       nn.LeakyReLU(negative_slope=0.1,inplace=True),
   )
   self.scale1_output = nn.Conv2d(256, (3*(4+1+self.class_n)), 1, 1, 1)



   

   #Global Average Poolingとして用いる。 (darknetのネットワーク)
   #self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))

 def forward(self, x):
   x = self.first_block(x)

   x = self.res_block1(x)
   x = self.conv1(x)

   x = self.res_block2(x)
   x = self.conv2(x)

   x = self.res_block3(x)
   scale1 = x
   x = self.conv3(x)

   x = self.res_block4(x)
   scale2 = x
   x = self.conv4(x)

   x = self.res_block5(x)
   x = self.ConvBlock(x)
  #  for layer in self.conv_block:
  #    x = layer(x)
   scale2_2 = x
   scale3 = x


   return x

  






In [25]:

x.shape

NameError: ignored